In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "data/scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)
len(box_scores)

4213

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [5]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [6]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score

In [7]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [8]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]

In [10]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

/var/folders/19/rwr1k24s12d163w7r0bm_zy00000gn/T/ipykernel_52181/585192420.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs = {"id": "line_score"})[0]
/var/folders/19/rwr1k24s12d163w7r0bm_zy00000gn/T/ipykernel_52181/385819283.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
/var/folders/19/rwr1k24s12d163w7r0bm_zy00000gn/T/ipykernel_52181/385819283.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
/v

KeyError: "['gmsc', 'gmsc_max'] not in index"

In [11]:
games_df = pd.concat(games, ignore_index=True)
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,41.0,84.0,0.488,12.0,38.0,0.316,23.0,25.0,...,30.8,31.1,165.0,122.0,ATL,109,1,None,2023-11-11,True
1,240.0,240.0,38.0,87.0,0.437,18.0,46.0,0.391,15.0,18.0,...,22.7,23.7,170.0,121.0,MIA,117,0,None,2023-11-11,False


In [69]:
full_game

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,41.0,84.0,0.488,12.0,38.0,0.316,23.0,25.0,...,30.8,31.1,165.0,122.0,ATL,109,1,None,2023-11-11,True
1,240.0,240.0,38.0,87.0,0.437,18.0,46.0,0.391,15.0,18.0,...,22.7,23.7,170.0,121.0,MIA,117,0,None,2023-11-11,False


In [35]:
advanced

,MP,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
Starters,,,,,,,,,,,,,,,,
Trae Young,NaN,0.566,0.447,0.474,0.579,0.0,6.2,2.9,46.9,1.3,0.0,17.3,31.1,120.0,121.0,2.4
De'Andre Hunter,NaN,0.421,0.364,0.364,0.182,0.0,16.9,7.8,8.3,1.4,2.9,20.2,17.3,80.0,117.0,-8.7
Dejounte Murray,NaN,0.700,0.719,0.563,0.063,0.0,16.9,7.8,26.1,4.1,2.9,15.4,22.6,125.0,110.0,11.0
Clint Capela,NaN,0.667,0.667,0.000,0.000,29.9,25.1,27.7,0.0,2.0,8.7,0.0,10.4,165.0,110.0,3.8
Jalen Johnson,NaN,0.563,0.563,0.375,0.000,0.0,22.1,10.2,15.2,0.0,0.0,27.3,21.0,88.0,121.0,-6.7
Bogdan Bogdanović,NaN,0.577,0.577,0.923,0.000,0.0,12.5,5.8,5.6,0.0,0.0,13.3,21.6,104.0,122.0,-4.6
Saddiq Bey,NaN,0.444,0.444,0.778,0.000,12.5,24.6,18.1,0.0,0.0,0.0,30.8,22.0,72.0,120.0,-10.5
Onyeka Okongwu,NaN,0.258,0.000,0.000,0.667,17.0,15.0,16.0,10.5,0.0,8.7,0.0,6.7,129.0,116.0,-2.3
Wesley Matthews,NaN,1.215,1.500,1.000,1.000,13.6,16.0,14.7,0.0,0.0,0.0,25.8,21.5,129.0,122.0,11.8
